<a href="https://colab.research.google.com/github/axelpuyo/liasd/blob/master/explainers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip -q install import-ipynb

# # Uncomment this if you need to test functions inside this file itself.
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# print(os.getcwd())
# print(os.listdir())

# %cd drive/MyDrive/Colab Notebooks/liasd

Mounted at /content/drive
/content
['.config', 'drive', 'sample_data']
/content/drive/MyDrive/Colab Notebooks/liasd
importing Jupyter notebook from utils.ipynb


In [ ]:
import import_ipynb
import utils
import numpy as np
import tensorflow as tf

from tensorflow import keras

# Deep Explainers
See report for RISE, Grad-CAM and SHAP papers.

In [ ]:
def rise(model, image, label, num_masks, proba):
  '''
  In short, RISE takes an image and generates 'num_masks' binary occlusion masks. It multiplies this original image by these masks, and computes the difference in prediction accuracy for each occluded image.
  Averaging these differences over how many times each pixel of the image was present in a mask (thus occluded), it effectively provides a heatmap showing the individual contribution of each pixel to the accuracy of the classification of the original image.

  proba: (float) determines the amount of occlusion of the mask (0 = fully occluded mask, 1 = transparent mask)
  '''
  batch_image, batch_label = image[np.newaxis], label[np.newaxis]
  old_loss, old_accuracy = model.evaluate(batch_image, batch_label, verbose=0) # batch size = 1
  if not old_loss:
    old_loss += 1e-15

  masks = np.zeros((num_masks, image.shape[0], image.shape[1]))
  counts = np.zeros(masks.shape)
  masked_image = np.zeros(image.shape)
  for n in range(num_masks):
    masks[n] = np.kron(np.random.choice([0, 1], size=(int(np.floor(image.shape[0]/2)), int(np.floor(image.shape[1]/2))), p=[proba, 1 - proba]), np.ones((2,2)))
    for k in range(3):
      masked_image[..., k] = image[..., k]*masks[n]

    masked_batch_image = masked_image[np.newaxis]
    new_loss = model.evaluate(masked_batch_image, batch_label, verbose=0)[0]
    indexes = (masks[n] == 0)
    counts[n, indexes == True] = new_loss
  
  heatmap = np.average(counts, axis=0)

  # Print the loss to see how good the prediction on this image was
  print('Accuracy : ', old_accuracy)
  
  # Plots
  utils.image_plots(label, image, heatmap)
  
  return heatmap


In [ ]:
def gradcam(model, layer_name, image, label, pred_index=None, plots=True, verbose=True):
  ## Create separate models for the convolutional block (feature extraction) and the dense layer block (classification)
  # Feature extraction model
  if not tf.executing_eagerly():
    print('TensorFlow 2.0 behavior disabled. Please restart runtime.')
    return

  grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )
  
  # grad_model.layers[-1].activation = None
  
  batch_image, batch_label = image[np.newaxis], label[np.newaxis]
  
  ## Gradient extraction
  with tf.GradientTape() as tape:
        features, preds = grad_model(batch_image)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

  # Last gradient extraction
  grads = tape.gradient(class_channel, features)
  pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

  # Multiply features by "how important" it is wrt. the gradient
  features = features.numpy()[0]
  pooled_grads = pooled_grads.numpy()

  heatmap = tf.squeeze(features @ pooled_grads[..., tf.newaxis])
  heatmap = np.maximum(heatmap + 1e-13, 0) / np.max(heatmap + 1e-13)
  
  # Print the accuracy to see how good the prediction on this image was
  acc = model.evaluate(batch_image, batch_label, verbose=0)[1]
  if verbose:
    print(f'Accuracy : {acc}')

  # Plots
  if plots:
    utils.image_plots(label, image, heatmap)

  return heatmap, acc

In [ ]:
def shap(model, images, labels, num_background, num_test):
  !pip -q install shap
  import shap
  # tf.compat.v1.disable_v2_behavior()
  # tf.compat.v1.disable_eager_execution()
  if tf.executing_eagerly():
    print('TensorFlow 2.0 behavior still enabled. Please restart runtime.')
    return 

  background = images[np.random.choice(images.shape[0], num_background)]

  # This loop provides SHAP with 1 representative of each unique label in labels
  c = 0
  imgs = np.zeros((num_test, images.shape[1], images.shape[2], images.shape[3]))
  while c < num_test:
    idx = np.random.randint(0, 2000)
    if utils.uncategoric(labels[idx])[1] == c:
      imgs[c] = images[idx]
      c += 1

  e = shap.DeepExplainer(model, background)
  shap_values = e.shap_values(imgs)
  shap.image_plot(shap_values, imgs)

  return shap_values, imgs